In [26]:
import sys
sys.path.append("/home/raulteixeira/repos/CSPZ/scripts/")
import NoiseSOM as ns
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import scipy
import sys
import time

hdf_ = h5py.File('/project2/chihway/data/decade/metacal_test_20230427.hdf')

N=int(2e6)
with h5py.File('/project2/chihway/data/decade/metacal_test_20230427.hdf') as f:
    
    flux_r, flux_i, flux_z = np.array(f['mcal_flux_noshear'][:N]).T
    flux_err_r, flux_err_i, flux_err_z = np.array(f['mcal_flux_err_noshear'][:N]).T
    mag_r = 30 - 2.5*np.log10(flux_r)
    mag_i = 30 - 2.5*np.log10(flux_i)
    mag_z = 30 - 2.5*np.log10(flux_z)

    mcal_pz_mask = ((mag_i < 23.5) & (mag_i > 18) & 
                    (mag_r < 26)   & (mag_r > 15) & 
                    (mag_z < 26)   & (mag_z > 15) & 
                    (mag_r - mag_i < 4)   & (mag_r - mag_i > -1.5) & 
                    (mag_i - mag_z < 4)   & (mag_i - mag_z > -1.5))

    SNR     = np.array(f['mcal_s2n_noshear'][:N])
    T_ratio = np.array(f['mcal_T_ratio_noshear'][:N])
    T       = np.array(f['mcal_T_noshear'][:N])
    flags   = np.array(f['mcal_flags'][:N])
    
    g1, g2  = np.array(f['mcal_g_noshear'][:N]).T

    #Metacal cuts based on DES Y3 ones (from here: https://des.ncsa.illinois.edu/releases/y3a2/Y3key-catalogs)
    SNR_Mask   = (SNR > 10) & (SNR < 1000)
    Tratio_Mask= T_ratio > 0.5
    T_Mask     = T < 10
    Flag_Mask  = flags == 0

    Other_Mask = np.invert((T > 2) & (SNR < 30)) & np.invert((np.log10(T) < (22.25 - mag_r)/3.5) & (g1**2 + g2**2 > 0.8**2))

    Mask = mcal_pz_mask & SNR_Mask & Tratio_Mask & T_Mask & Flag_Mask & Other_Mask
    
    #These are the fluxes with all metacal cuts applied
    flux_r = flux_r[Mask]
    flux_i = flux_i[Mask]
    flux_z = flux_z[Mask]
    flux_err_r = flux_err_r[Mask]
    flux_err_i = flux_err_i[Mask]
    flux_err_z = flux_err_z[Mask]

fluxes_d = np.array([flux_r, flux_i, flux_z]).T
fluxerrs_d = np.array([flux_err_r, flux_err_i, flux_err_z]).T

def flux2mag(flux):
    return -2.5 * np.log10(flux) + 30

bands = ['r', 'i', 'z']

cols = [f'flux_{band}' for band in bands]+[f'flux_err_{band}' for band in bands]
df = pd.DataFrame(np.hstack([fluxes_d, fluxerrs_d]), columns=cols)

mags_d = np.zeros((len(df),len(bands)))
magerrs_d = np.zeros((len(df),len(bands)))

for i,band in enumerate(bands):
    print(i,band)
    mags_d[:,i] = flux2mag(fluxes_d[:,i])

colors = np.zeros((len(fluxes_d),len(bands)-1))
for i in range(len(bands)-1):
    colors[:,i] = mags_d[:,i] - mags_d[:,i+1]

normal_colors = np.mean(colors > -1, axis=1) == 1
normal_colors.sum()


df = df[normal_colors]

#mask faint objects, i < 25
df = df[flux2mag(df.flux_i.values) < 25]

# g = flux2mag(df.BDF_FLUX_DERED_CALIB_G.values)
# i = flux2mag(df.BDF_FLUX_DERED_CALIB_I.values)
# r = flux2mag(df.BDF_FLUX_DERED_CALIB_R.values)
# z = flux2mag(df.BDF_FLUX_DERED_CALIB_Z.values)
#k = flux2mag(df.BDF_FLUX_DERED_CALIB_KS.values)


# plt.hexbin((r-z), (z-k), gridsize=100, mincnt=1, bins='log')
# _t = np.linspace(-1, 5, 100)
# plt.plot(_t, 0.5*_t, color='r', ls='--')
# plt.xlabel(r"$r-z$")
# plt.ylabel(r"$z-K$")
# plt.show()

# mask stars based on (z−K) > 0.5×(r −z) color cut

# df = df[(z-k) > 0.5*(r-z)]

data=df

data.loc[:,"mag_i"]=flux2mag(data.loc[:, 'flux_i'])

def balrog_sigmoid(x, x0):
    """Sigmoid function
    Parameters
    ----------
    x : float or array-like
        Points at which to evaluate the function.
    x0 : float or array-like
        Location of transition.
    Returns
    -------
    sigmoid : scalar or array-like, same shape as input
    """
    return 1.0 - 1.0 / (1.0 + np.exp(-4.0 * (x - x0)))

def mock_balrog_sigmoid(
    deep_data, 
    sigmoid_x0,
    N,
    ref_mag_col = "mag_i"
):
    """
    Function for selecting deep field galaxies at a rate that follows a sigmoid function that smoothly transitions from 1 for bright objects, to a value of 0 for faint objects. 
    Parameters
    ----------
    deep_data : pandas dataframe
        Pandas dataframe containing the deep field data.
    sigmoid_x0 : float
        Magnitude value at which the sigmoid function transitions from 1 to 0.
    N : int
        Number of galaxies to be drawn.
    ref_mag_col : string
        Column name of the reference magnitude in deep_data
    Returns
    -------
    deep_balrog_selection : pandas dataframe
        Pandas dataframe containing a list of N deep field objects to be injected by Balrog.
    """
    np.random.seed()
    mag_ref = deep_data.loc[:, ref_mag_col].values
    weights = balrog_sigmoid(mag_ref, sigmoid_x0)
    weights/=sum(weights)
    selected_objects = np.random.choice(len(deep_data), N, p=weights, replace=True)
    
    deep_balrog_selection = deep_data.iloc[selected_objects]
    return deep_balrog_selection
                          
    
# New Pandas Dataframe with only detected galaxies
# A value of 23.0 returns something similar to Balrog Y3
# A value of 23.5 maybe is similar to the WL sample in Y6.
# A value of 21.5 is maybe optimal for LSS samples in Y6.
#balrog_mocked = mock_balrog_sigmoid(deep_data, 23.0, nTrain)

nTrain = len(data)
balrog_mocked = mock_balrog_sigmoid(data, 23.0, nTrain)

fluxes_d = np.zeros((len(balrog_mocked),len(bands)))
fluxerrs_d = np.zeros((len(balrog_mocked),len(bands)))

for i,band in enumerate(bands):
    print(i,band)
    fluxes_d[:,i] = balrog_mocked['flux_%s'%band]
    fluxerrs_d[:,i] = balrog_mocked['flux_err_%s'%band]

# Train the SOM with this set (takes a few hours on laptop!)
#len(fluxes_d)

# Scramble the order of the catalog for purposes of training
#indices = np.random.choice(nTrain, size=nTrain, replace=False)
indices = np.random.choice(nTrain, size=nTrain, replace=False)
hh = ns.hFunc(nTrain, sigma=(30,1))
metric = ns.AsinhMetric(lnScaleSigma=0.4, lnScaleStep=0.03)

/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log10
/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in log10
/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in log10
/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in log10


0 r
1 i
2 z
0 r
1 i
2 z


In [27]:
nTrain

993478

In [1]:
%time
som = ns.NoiseSOM(metric, fluxes_d[indices,:], fluxerrs_d[indices,:], \
    learning=hh, \
    shape=(48,48), \
    wrap=False,logF=True, \
    initialize='sample', \
    minError=0.02)

path_cats='/project2/chihway/raulteixeira/data/'
# And save the resultant weight matrix
np.save("%s/som_delve_48_48.npy"%path_cats,som.weights)